# [2020年3月] CRBMによる異常検知

CRBMを用いて異常データの検知を行う。

## 必要なモジュールのimport

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
import time

## 条件付き制限ボルツマンマシン (CRBM)

In [2]:
class CRBM(object):
    def __init__(self, input_data=None, input_history=None, n_hidden=100, input_dim=1, delay=6, A=None, B=None, W=None, hbias=None, vbias=None):

        self.input_dim = input_dim  # 入力層データの次元数
        self.n_hidden = n_hidden  # 隠れ層のユニット数
        self.delay = delay # 遡るデータ数

        if A is None: # [入力層-出力層] 間の重みの初期化
            A = np.array(np.random.uniform(low = -0.01, high = 0.01, size = (input_dim * delay, input_dim)))

        if B is None: # [入力層-隠れ層] 間の重みの初期化
            B = np.array(np.random.uniform(low = -0.01, high = 0.01, size = (input_dim * delay, n_hidden)))

        if W is None: # [出力層-隠れ層] 間の重みの初期化
            W = np.array(np.random.uniform(low = -0.01, high = 0.01, size = (input_dim, n_hidden)))

        if hbias is None: # 隠れ層のバイアスの初期化
            hbias = np.zeros(n_hidden)

        if vbias is None: # 出力層のバイアスの初期化
            vbias = np.zeros(input_dim)

        # インスタンス変数の作成
        self.input_data = input_data
        self.input_history = input_history
        self.A = A
        self.B = B
        self.W = W
        self.hbias = hbias
        self.vbias = vbias

    # シグモイド関数 (OverFlow対策有り)
    def _sigmoid(self, x):
        sigmoid_range = 34.538776394910684
        x = np.clip(x, -sigmoid_range, sigmoid_range)
        return 1.0 / (1.0 + np.exp(-x))

    # CD法 (lr:学習率, k:反復回数)
    def contrastive_divergence(self, lr=0.1, k=1, input_data=None, input_history=None):

        if input_data is not None: # 出力層のデータ
            self.input_data = input_data

        if input_history is not None: # 入力層のデータ
            self.input_history = input_history

        # 隠れ層のサンプリング
        ph_mean, ph_sample = self.sample_h_given_v(self.input_data, self.input_history)
        nh_sample = ph_mean # 期待値がサンプル結果 (本来は0か1だが期待値を用いた方が結果が良くなる)

        # ギブスサンプリングの往復
        for step in range(k):
            nv_mean, nv_sample, nh_mean, nh_sample = self.gibbs_hvh(nh_sample, self.input_history)
            nh_sample = nh_mean # 隠れ層のサンプリング結果として期待値を用いる

        # パラメータの更新
        self.vbias += lr * np.mean(self.input_data - nv_sample, axis=0)
        self.hbias += lr * np.mean(ph_sample - nh_sample, axis=0)
        self.A += lr * (self.input_history.T @ (self.input_data - nv_sample))
        self.B += lr * (self.input_history.T @ (ph_sample - nh_sample))
        self.W += lr * (self.input_data.T @ ph_sample - nv_sample.T @ nh_sample)

    # 隠れ層の期待値
    def propup(self, v, v_history):
        return self._sigmoid(v @ self.W + v_history @ self.B + self.vbias)

    # 隠れ層のサンプリング
    def sample_h_given_v(self, v0_sample, v_history):
        h1_mean = self.propup(v0_sample, v_history) # 状態値の期待値
        h1_sample = np.random.binomial(size = h1_mean.shape, n=1, p=h1_mean) # 二項分布から期待値に従ってサンプリング
        return [h1_mean, h1_sample] # [期待値, 二値]

    # 出力層の期待値
    def propdown(self, h, v_history):
        mean_activation = h @ self.W.T + v_history @ self.A + self.vbias # ガウス分布の平均値(=期待値)
        return mean_activation

    # 出力層のサンプリング
    def sample_v_given_h(self, h0_sample, v_history):
        v1_mean = self.propdown(h0_sample, v_history)
        v1_sample = v1_mean # 期待値をサンプリング結果として用いる
        return [v1_mean, v1_sample] # [期待値, 期待値]

    # ギブスサンプリング
    def gibbs_hvh(self, h0_sample, v_history):
        v1_mean, v1_sample = self.sample_v_given_h(h0_sample, v_history)
        h1_mean, h1_sample = self.sample_h_given_v(v1_sample, v_history)
        return [v1_mean, v1_sample, h1_mean, h1_sample]

    # 出力値の生成
    def recon(self, v_history, k=None, threshold=1e-3):
        v = np.zeros(self.input_dim)

        if k is None:
            k = 5

        for _ in range(k):
            h_mean, h_sample = self.sample_h_given_v(v, v_history)
            _, nv = self.sample_v_given_h(h_mean, v_history)

            if np.abs(nv - v).all() < threshold:
                break

            v = nv

        return v

    # 誤差関数
    def mean_squared_error(self, v, v_history):
        return np.sum((v - self.recon(v_history,k=5)) **2)

    # 条件付き確率 P( v | v_history ) の計算関数
    #     ※注意: データが2次元以上の場合コード修正の必要あり
    def conditional_dist(self, v, v_history):
        #第一項
        temp1 = np.exp(-0.5 * (v - self.vbias - v_history@self.A)**2) # 入力データが標準化されているので、分散σ = 1 と置いている
        #第二項
        temp2 = np.prod(1 + np.exp(self.hbias + v_history@self.B + v*self.W))
        return temp1*temp2

    # 学習用関数
    def fit(self, X_data, y_data, batch_size, epochs, validation_split, learning_rate=1e-3):

        costs = [] # 学習途中の誤差のログ
        cnt = 0
        last_cost = 0

        X_train, X_val, y_train, y_val = train_test_split(X_data,y_data,test_size=validation_split) # 学習データと検証データの分割

        for _ in tqdm(range(epochs)):

            # batch_size の数だけ学習データからサンプリング
            batch_mask = np.random.choice(X_train.shape[0], batch_size)
            X_batch = X_train[batch_mask]
            y_batch = y_train[batch_mask]

            # CD法による学習
            self.contrastive_divergence(lr=learning_rate, k=1, input_data=y_batch, input_history=X_batch)

            # 誤差のログの追加
            cost = self.mean_squared_error(y_val, X_val)
            costs.append(cost)

            # validationデータに対する誤差が5回連続で増えると打ち切り
            cnt = cnt+1 if cost > last_cost else 0
            last_cost = cost
            if cnt >= 5:
              break

        # 誤差の表示
        plt.plot(costs)
        plt.show()

    def param(self):
        return self.A, self.B, self.W, self.hbias, self.vbias

## データ読み込み

### 学習済みパラメータの読み込み

In [3]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

# 読み込み元パス
%cd /content/drive/My Drive/research

# 保存
A = np.loadtxt('./crbm_param_A.csv', delimiter=',')
B = np.loadtxt('./crbm_param_B.csv', delimiter=',')
W = np.loadtxt('./crbm_param_W.csv', delimiter=',')
hbias = np.loadtxt('./crbm_param_hbias.csv', delimiter=',')
vbias = np.loadtxt('./crbm_param_vbias.csv', delimiter=',')

Mounted at /content/drive
[Errno 2] No such file or directory: '/content/drive/My Drive/research'
/content


FileNotFoundError: ./crbm_param_A.csv not found.

## CRBMの構成

In [ ]:
delay = B.shape[0]
crbm = CRBM(input_data=None, input_history=None, n_hidden=10,
             input_dim=1, delay=delay, A=A, B=B, W=W, hbias=hbias, vbias=vbias)

## データセットAの評価

### 前処理済みデータの読み込み

In [ ]:
# 読み込み元パス
%cd /content/drive/My Drive/research

test_data_X = np.loadtxt('./test_data_A_X.csv', delimiter=',')
test_data_y = np.loadtxt('./test_data_A_y.csv', delimiter=',')

size, delay = test_data_X.shape
test_data_y = test_data_y.reshape(size,1)

### 前処理前データの読み込み

In [ ]:
# 読み込みパス
%cd /content/drive/My Drive/research

# 読み込み
test_data = np.loadtxt('./test_data_A.csv', delimiter=',')
test_log = np.loadtxt('./test_log_A.csv', delimiter=',')

In [ ]:
print(test_data_X.shape)
print(test_data_y.shape)
print(test_data.shape)
print(test_log.shape)

In [ ]:
test_data_y.mean()


In [ ]:
test_data.mean()
test_data.std()

In [ ]:
test_data

In [ ]:
test_data.shape

In [ ]:
test_log

In [ ]:
test_log.shape

In [ ]:
num_in_1 = []
for i in range(5000):
  if np.all(test_log[i]==0):
    continue
  else:
    num_in_1.append(i)
print(num_in_1)

In [ ]:
num_in_1 = np.array(num_in_1)
num_in_1.shape

In [ ]:
for i in num_in_1:
  print(np.count_nonzero(test_log[i]==1))

### ROC曲線

#### 閾値の探索範囲を決定

In [ ]:
seq_len = 125 - delay # シーケンスの長さ
seq_num = int(size / seq_len )# シーケンスの個数

Anomaly_rate = []
for seq in range(seq_num):

    # シーケンスに区切った前処理済みデータ
    seq_X = test_data_X[seq*seq_len : (seq+1)*seq_len]
    seq_y = test_data_y[seq*seq_len : (seq+1)*seq_len]

    for i in range(seq_len):
      # 条件付き確率を計算
      c_dist = crbm.conditional_dist(seq_y[i], seq_X[i])

      # 条件付き確率の負の対数を異常度として計算
      Anomaly_rate.append(-np.log(c_dist))

Anomaly_rate = np.array(Anomaly_rate).reshape(seq_num, seq_len)

th_max = np.max(Anomaly_rate) # 閾値の最大値
th_min = np.min(Anomaly_rate) # 閾値の最小値

print("閾値最大 = {}".format(th_max))
print("閾値最小 = {}".format(th_min))

In [ ]:
seq_len = 125 - delay # シーケンスの長さ
seq_num = int(size / seq_len )# シーケンスの個数

Anomaly_rate = []
seq = 0

# シーケンスに区切った前処理済みデータ
seq_X = test_data_X[seq*seq_len : (seq+1)*seq_len]
seq_y = test_data_y[seq*seq_len : (seq+1)*seq_len]

for i in range(seq_len):
  # 条件付き確率を計算
  c_dist = crbm.conditional_dist(seq_y[i], seq_X[i])
  Anomaly_rate.append(c_dist)

plt.plot(Anomaly_rate)


In [ ]:
print(seq_num)

#### 座標を計算

In [ ]:
# 閾値最小から最大までの区間を区切って異常検知
threshold = np.linspace(th_min-1e-4, th_max+1e-4, 100)

In [ ]:
print(threshold)

In [ ]:
# 最良な閾値
best_th = 0
temp = 0

# ROC曲線の座標
X = []
Y = []

for th in tqdm(threshold):
  TP, TN, FP, FN = 0, 0, 0, 0
  for seq in range(seq_num):
    d_step = -1
    flag = False
    for i in range(seq_len):
      if test_log[seq][i+delay] == 1:
        flag = True
      if Anomaly_rate[seq][i] >= th:
        d_step = i
        break

    if d_step == -1: # 異常無しと判断
      if flag == False: # 異常なしで異常なしと判断
        TN += 1
      else: # 異常ありで異常なしと判断
        FN += 1

    else: # 異常ありと判断
      # 異常ありと判断した以前、5step以内に異常が起きているか
      if flag == True: # 異常ありで異常ありと判断
        TP += 1
      else: # 異常なしで異常ありと判断
        FP += 1

  print("TP : {}, TN : {}, FP : {}, FN : {}, th:{}".format(TP,TN,FP,FN,th))
  # ROC曲線のX,Y座標
  Y.append(TP/(TP+FN) if (TP+FN) != 0 else 0) # 真陽性率
  X.append(FP/(FP+TN) if (FP+TN) != 0 else 0) # 偽陽性率

  # 最良な閾値の探索
  if temp < (Y[-1]-X[-1]):
    best_th = th
    temp = Y[-1]-X[-1]

print(best_th)

#### 描画

In [ ]:
fig = plt.figure(figsize=(4,4))
fig = plt.scatter(X ,Y)
fig = plt.xlim(0,1)
fig = plt.ylim(0,1)

### AUCの計算

In [ ]:
X = np.array(X)
Y = np.array(Y)

# X軸方向に座標をソート
sorted_X = X[X.argsort()]
sorted_Y = Y[X.argsort()]

AUC = 0.0

for i in range(1,len(sorted_X)):
    if sorted_X[i] == sorted_X[i-1]:
        sorted_Y[i] = max(sorted_Y[i],sorted_Y[i-1])
    AUC += (sorted_Y[i]+sorted_Y[i-1])*(sorted_X[i]-sorted_X[i-1])
AUC /= 2

print(AUC)

### 異常検知結果の可視化

In [ ]:
%cd /content/drive/My Drive/research

visual_num = 5

seq_len = 125 - delay # シーケンスの長さ
seq_num = int(size / seq_len) # シーケンスの個数

fig = plt.figure(figsize=(10,10))

for vi in range(visual_num):

    # ランダムなシーケンス番号
    rand_seq = np.random.randint(0, seq_num)

    # シーケンス番号に対応した前処理済みデータ
    seq_X = test_data_X[rand_seq*seq_len : (rand_seq+1)*seq_len]
    seq_y = test_data_y[rand_seq*seq_len : (rand_seq+1)*seq_len]

    f_log = []
    d_step = -1e3

    for i in range(seq_len):
      # 条件付き確率を計算
      c_dist = crbm.conditional_dist(seq_y[i], seq_X[i])

      # 条件付き確率の負の対数を異常度として計算
      a_rate = -np.log(c_dist)
      f_log.append(a_rate)
      # 異常と判断されたところで異常度の計算を終える
      if(best_th < a_rate):
        d_step = i # 異常判断をした時点
        break

    # シーケンス番号に対応した前処理前データ
    d = test_data[rand_seq]

    # 前処理前データのプロット
    fig = plt.subplot(visual_num, 2, 2*vi+1)
    fig = plt.plot(np.arange(76,201),d, c="black")
    fig = plt.plot([d_step+delay+76,d_step+delay+76],[-1e9,1e9],c="red")
    for j, flag in enumerate(test_log[rand_seq]):
      if flag == 1:
        fig = plt.scatter(j+1+75, test_data[rand_seq][j], c="red")
    fig = plt.xlim(75,200)
    fig = plt.ylim(-0.5,0.5)

    # 異常度のプロット
    fig = plt.subplot(visual_num, 2, 2*vi+2)
    fig = plt.plot(np.arange(delay, len(f_log)+delay)+75,f_log,c="black")
    fig = plt.plot([d_step+delay+76,d_step+delay+76],[-1e9,1e9],c="red")
    for j, flag in enumerate(test_log[rand_seq]):
      if flag == 1:
        fig = plt.scatter(j+75, f_log[j-delay], c="red")
    fig = plt.xlim(75,200)
    fig = plt.ylim(-7,5)

    #fig = plt.ylim(-100,50)